# Bölüm 3: Rol Atama (Role Prompting)

- [Ders](#lesson)
- [Egzersizler](#exercises)
- [Örnek Deneme Alanı](#example-playground)

## Kurulum

API anahtarınızı yüklemek ve `get_completion` yardımcı fonksiyonunu oluşturmak için aşağıdaki kurulum hücresini çalıştırın.

In [ ]:
%pip install anthropic --quiet

# utils paketinden hints modülünü içe aktarın
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# Python'un yerleşik düzenli ifade (regular expression) kütüphanesini içe aktarın
import re
from anthropic import AnthropicBedrock

%store -r MODEL_NAME
%store -r AWS_REGION

client = AnthropicBedrock(aws_region=AWS_REGION)

def get_completion(prompt, system=''):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": prompt}
        ],
        system=system
    )
    return message.content[0].text

---

## Ders

Claude'un, sizin söyledikleriniz dışında hiçbir bağlama (context) sahip olmadığı temasına devam edersek; bazen **Claude'u belirli bir role bürünmesi için yönlendirmek (gerekli tüm bağlamı dahil ederek)** önemlidir. Bu yöntem **"Rol Atama" (Role Prompting)** olarak da bilinir. Rol bağlamındaki detay ne kadar fazlaysa, sonuç o kadar iyi olur.

**Claude'u bir rolle hazırlamak (priming)**; yazarlıktan kodlamaya, özetlemeden analize kadar pek çok alanda **Claude'un performansını artırabilir**. Bu, tıpkı insanlara "bir ______ gibi düşün" denildiğinde verilen desteğe benzer. Rol atama; Claude'un yanıtının stilini, tonunu ve tarzını da değiştirebilir.

**Not:** Rol atama, sistem promptunda (system prompt) veya kullanıcı mesajı (User message) turunun bir parçası olarak gerçekleştirilebilir.

### Örnekler

Aşağıdaki örnekte, rol atama (role prompting) olmadan Claude'un kaykay konusuna tek cümlelik bir bakış açısı sunması istendiğinde, **doğrudan ve üslupsuz (non-stylized) bir cevap** verdiğini görüyoruz.

Ancak Claude'u bir kedi rolüne bürünmesi için hazırladığımızda, Claude'un bakış açısı değişir ve böylece **Claude'un yanıt tonu, stili ve içeriği yeni role uyum sağlar**. 

**Not:** Kullanabileceğiniz ek bir teknik ise **Claude'a hedef kitlesi hakkında bağlam sunmaktır**. Aşağıdaki örneği, Claude'a kiminle konuştuğunu da söyleyecek şekilde düzenleyebilirdik. "Sen bir kedisin" talimatı, "Sen bir grup kaykaycıya hitap eden bir kedisin" talimatından oldukça farklı bir sonuç üretir.

İşte sistem promptunda rol atama yapılmamış haliyle ilgili prompt:

In [ ]:
# Prompt
PROMPT = "Tek bir cümleyle, kaykay yapmak hakkında ne düşünüyorsun?"

# Claude'un yanıtını yazdır
print(get_completion(PROMPT))

İşte aynı kullanıcı sorusu, ancak bu sefer rol atama (role prompting) yapılmış haliyle

In [ ]:
# Sistem promptu
SYSTEM_PROMPT = "Sen bir kedisin."

# Prompt
PROMPT = "Tek bir cümleyle, kaykay yapmak hakkında ne düşünüyorsun?"

# Claude'un yanıtını yazdır
print(get_completion(PROMPT, SYSTEM_PROMPT))

Rol atamayı; Claude'un belirli yazı stillerini taklit etmesini sağlamak, belirli bir ses tonuyla konuşmasını sağlamak veya yanıtlarının karmaşıklığını yönlendirmek için bir yöntem olarak kullanabilirsiniz. **Rol atama, Claude'un matematik veya mantık görevlerini yerine getirme konusundaki becerisini de artırabilir.**

Örneğin, aşağıdaki örnekte "evet" olan kesin bir doğru cevap vardır. Ancak Claude yanlış yapar ve elinde bilgi eksikliği olduğunu düşünür; oysa tüm bilgiler oradadır:

In [ ]:
# Prompt
PROMPT = "Jack, Anne'e bakıyor. Anne, George'a bakıyor. Jack evli, George evli değil ve Anne'nin evli olup olmadığını bilmiyoruz. Evli bir kişi, evli olmayan bir kişiye mi bakıyor?"

# Claude'un yanıtını yazdır
print(get_completion(PROMPT))

Peki ya **Claude'u bir mantık botu (logic bot) olarak hareket etmesi için hazırlarsak** ne olur? Bu, Claude'un cevabını nasıl değiştirecek?

Görünüşe göre bu yeni rol atamasıyla Claude soruyu doğru biliyor. (Her ne kadar sunduğu gerekçelerin tamamı tam olarak doğru olmasa da.)

In [ ]:
# Sistem promptu
SYSTEM_PROMPT = "Karmaşık mantık problemlerini yanıtlamak için tasarlanmış bir mantık botusun."

# Prompt
PROMPT = "Jack, Anne'e bakıyor. Anne, George'a bakıyor. Jack evli, George evli değil ve Anne'nin evli olup olmadığını bilmiyoruz. Evli bir kişi, evli olmayan bir kişiye mi bakıyor?"

# Claude'un yanıtını yazdır
print(get_completion(PROMPT, SYSTEM_PROMPT))

**Not:** Bu kurs boyunca öğreneceğiniz şey, **benzer sonuçlar elde etmek için kullanabileceğiniz pek çok prompt mühendisliği tekniği** olduğudur. Hangi teknikleri kullanacağınız size ve tercihinize bağlıdır! Kendi prompt mühendisliği tarzınızı bulmanız için **denemeler yapmanızı** teşvik ediyoruz.

Yukarıdaki hiçbir içeriği değiştirmeden ders promptları ile denemeler yapmak isterseniz, [**Örnek Deneme Alanı**](#örnek-deneme-alanı) kısmını ziyaret etmek için ders not defterinin (notebook) en altına inin.

---

## Egzersizler
- [Egzersiz 3.1 - Matematik Düzeltmesi](#exercise-31---math-correction)

### Egzersiz 3.1 - Matematik Düzeltmesi

Bazı durumlarda **Claude matematik konusunda**, hatta basit matematik işlemlerinde bile **zorlanabilir**. Aşağıda Claude, ikinci adımda bariz bir aritmetik hata olmasına rağmen matematik probleminin doğru çözüldüğünü yanlış bir şekilde değerlendiriyor. Claude'un adım adım ilerlerken hatayı aslında fark ettiğine, ancak genel çözümün yanlış olduğu sonucuna hemen varmadığına dikkat edin.

Claude'un çözümü "doğru" (correctly) yerine **"yanlış" (incorrectly)** olarak derecelendirmesini sağlamak için `PROMPT` ve/veya `SYSTEM_PROMPT` kısmını düzenleyin.

In [ ]:
# Sistem promptu - bir sistem promptu kullanmak istemiyorsanız, bu değişkeni boş bir dize olarak bırakabilirsiniz
SYSTEM_PROMPT = ""

# Prompt
PROMPT = """Aşağıdaki denklem aşağıda doğru çözülmüş mü?

2x - 3 = 9
2x = 6
x = 3"""

# Claude'un yanıtını al
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Egzersiz doğruluğunu değerlendiren fonksiyon
def grade_exercise(text):
    if "incorrect" in text or "not correct" in text.lower() or "yanlış" in text.lower():
        return True
    else:
        return False

# Claude'un yanıtını ve karşılık gelen notu yazdır
print(response)
print("\n--------------------------- DEĞERLENDİRME ---------------------------")
print("Bu egzersiz doğru şekilde çözüldü:", grade_exercise(response))

❓ Bir ipucu isterseniz, aşağıdaki hücreyi çalıştırın!

In [ ]:
print(hints.exercise_3_1_hint)

### Tebrikler!

Buraya kadar olan tüm egzersizleri çözdüyseniz, bir sonraki bölüme geçmeye hazırsınız demektir. İyi promptlamalar!

---

## Örnek Deneme Alanı

Burası, bu derste gösterilen prompt örneklerini özgürce denemeniz ve yaptığınız küçük değişikliklerin Claude'un yanıtlarını nasıl etkilediğini görmeniz için ayrılmış bir alandır.

In [ ]:
# Prompt
PROMPT = "In one sentence, what do you think about skateboarding?"

# Claude'un yanıtını yazdır
print(get_completion(PROMPT))

In [ ]:
# Sistem prompt
SYSTEM_PROMPT = "You are a cat."

# Prompt
PROMPT = "In one sentence, what do you think about skateboarding?"

# Claude'un yanıtını yazdır
print(get_completion(PROMPT, SYSTEM_PROMPT))

In [ ]:
# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Claude'un yanıtını yazdır
print(get_completion(PROMPT))

In [ ]:
# Sistem prompt
SYSTEM_PROMPT = "You are a logic bot designed to answer complex logic problems."

# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Claude'un yanıtını yazdır
print(get_completion(PROMPT, SYSTEM_PROMPT))